In [2]:
import requests as re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
import os
import sys
import time
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

In [3]:
from platform import python_version

print(python_version())

3.9.6


In [4]:
def flatten(input):
    new_list = []
    for i in input:
        if not (isinstance(i, str)):
            for j in i:
                new_list.append(j)
        else:
            new_list.append(i)
    return new_list

df = pd.DataFrame()
while len(df) < 1000:
    if len(df) == 0:
        r = re.get('https://api.pushshift.io/reddit/search/submission/?subreddit=CryptoCurrency&size=100')
    else:
        r = re.get('https://api.pushshift.io/reddit/search/submission/?subreddit=CryptoCurrency&size=100&before=' + str(int(df.iloc[-1]['created_utc'])))
    if r.status_code == 200:
        r = pd.json_normalize(r.json(), record_path='data')[['author', 'created_utc', 'id', 'is_self', 'link_flair_text', 'num_comments', 'removed_by_category', 'score', 'selftext', 'title', 'url']]
        r = r.where((len(r['selftext']) > 1) & (r['selftext'] != '[removed]') & (r['selftext'] != '[deleted]') & (r['removed_by_category'].isna())).dropna(how='all')
        r['selftext'] = r['selftext'].str.split('.')
        r['data'] = r[['title', 'selftext']].values.tolist()
        r['data'] = r['data'].apply(flatten)
        df = pd.concat([df, r], ignore_index=True)
    else:
        time.sleep(15)
os.remove('posts.csv')
df.to_csv('posts.csv')

In [5]:
analyzer = SentimentIntensityAnalyzer()

def senticol(cols):
    scores = {'neg':[], 'neu':[], 'pos':[], 'compound':[]}
    for i in range(0, len(cols['data'])):
        score = analyzer.polarity_scores(cols['data'][i])
        scores['neg'].append(score['neg'])
        scores['neu'].append(score['neu'])
        scores['pos'].append(score['pos'])
        scores['compound'].append(score['compound'])
    return pd.Series([scores['neg'], scores['neu'], scores['pos'], scores['compound']])

texts = df
texts[['neg','neu','pos','compound']] = texts.apply(senticol, axis=1)
os.remove('newscores.csv')
texts.to_csv('newscores.csv')
texts

,author,created_utc,id,is_self,link_flair_text,num_comments,removed_by_category,score,selftext,title,url,data,neg,neu,pos,compound
0,TwistedFluke,1.628996e+09,p4ljkg,1.0,PERSPECTIVE,0.0,NaN,1.0,[We all thrive to make profit for a range of g...,What would you do if you hit big on crypto and...,https://www.reddit.com/r/CryptoCurrency/commen...,[What would you do if you hit big on crypto an...,"[0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.775, 0.906, 0.767, 0.0]","[0.0, 0.225, 0.094, 0.233, 0.0]","[0.0, 0.4404, 0.3612, 0.6682, 0.0]"
1,johnnylaw54,1.628996e+09,p4ljc9,0.0,MOONS 🌕,0.0,NaN,1.0,[],All hail VETHOR!,https://marketrealist.com/p/vethor-vtho-price-...,"[All hail VETHOR!, ]","[0.0, 0.0]","[0.557, 0.0]","[0.443, 0.0]","[0.1511, 0.0]"
2,cho0n22,1.628996e+09,p4lhcm,0.0,POLITICS,0.0,NaN,1.0,[],Are Crypto's High Trading Volumes A Scam?,https://finance.yahoo.com/m/e1945811-b6db-31eb...,"[Are Crypto's High Trading Volumes A Scam?, ]","[0.381, 0.0]","[0.619, 0.0]","[0.0, 0.0]","[-0.5719, 0.0]"
3,holyshithead,1.628995e+09,p4lg9e,1.0,SUPPORT,1.0,NaN,1.0,[I just got to thinking after hearing that Mar...,What if doge exists now only to ruin the crypt...,https://www.reddit.com/r/CryptoCurrency/commen...,[What if doge exists now only to ruin the cryp...,"[0.18, 0.0, 0.083, 0.135, 0.0, 0.0]","[0.711, 0.946, 0.917, 0.865, 0.803, 0.0]","[0.109, 0.054, 0.0, 0.0, 0.197, 0.0]","[-0.3612, 0.3612, -0.4019, -0.4215, 0.4019, 0.0]"
4,Lj3380,1.628995e+09,p4lg3p,1.0,NaN,0.0,NaN,1.0,"[Slow and steady, it looks like BAT is coming ...",BAT Token,https://www.reddit.com/r/CryptoCurrency/commen...,"[BAT Token, Slow and steady, it looks like BAT...","[0.0, 0.0, 0.0, 0.0]","[1.0, 0.848, 0.811, 0.813]","[0.0, 0.152, 0.189, 0.187]","[0.0, 0.3612, 0.4215, 0.5362]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,DoeyB,1.628950e+09,p48jqt,1.0,SELF-STORY,0.0,NaN,1.0,[Just thought id like to make a friendly remin...,The importance of multiple emails,https://www.reddit.com/r/CryptoCurrency/commen...,"[The importance of multiple emails, Just thoug...","[0.0, 0.048, 0.0, 0.0, 0.0]","[0.615, 0.836, 1.0, 0.805, 0.885]","[0.385, 0.117, 0.0, 0.195, 0.115]","[0.3612, 0.8934, 0.0, 0.4404, 0.6652]"
1030,kitisgreat,1.628950e+09,p48jqf,0.0,FINANCE,0.0,NaN,1.0,[],Surprising Investor: Intel Now Owns Coinbase S...,https://cryptopotato.com/surprising-investor-i...,[Surprising Investor: Intel Now Owns Coinbase ...,"[0.0, 0.0]","[0.538, 0.0]","[0.462, 0.0]","[0.5106, 0.0]"
1031,Caralynethegreat,1.628950e+09,p48is3,0.0,FINANCE,0.0,NaN,1.0,[],Nearly 30% of Canadians Are Cryptocurrency Inv...,https://cryptopotato.com/nearly-30-of-canadian...,[Nearly 30% of Canadians Are Cryptocurrency In...,"[0.0, 0.0]","[1.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]"
1032,dalinara,1.628950e+09,p48ijg,0.0,🔴 UNRELIABLE SOURCE,2.0,NaN,1.0,[],XRP chart triggers sell-off warning after pric...,https://cointelegraph.com/news/xrp-chart-trigg...,[XRP chart triggers sell-off warning after pri...,"[0.167, 0.0]","[0.833, 0.0]","[0.0, 0.0]","[-0.34, 0.0]"


In [6]:
#VADER
def rem_zero(arr):
    return [i for i in arr if i != 0]

texts['comp_mod'] = texts['compound'].apply(rem_zero)
texts['comp_means'] = texts['comp_mod'].apply(np.mean).fillna(0)
texts

C:\Users\tbone\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


,author,created_utc,id,is_self,link_flair_text,num_comments,removed_by_category,score,selftext,title,url,data,neg,neu,pos,compound,comp_mod,comp_means
0,TwistedFluke,1.628996e+09,p4ljkg,1.0,PERSPECTIVE,0.0,NaN,1.0,[We all thrive to make profit for a range of g...,What would you do if you hit big on crypto and...,https://www.reddit.com/r/CryptoCurrency/commen...,[What would you do if you hit big on crypto an...,"[0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.775, 0.906, 0.767, 0.0]","[0.0, 0.225, 0.094, 0.233, 0.0]","[0.0, 0.4404, 0.3612, 0.6682, 0.0]","[0.4404, 0.3612, 0.6682]",0.489933
1,johnnylaw54,1.628996e+09,p4ljc9,0.0,MOONS 🌕,0.0,NaN,1.0,[],All hail VETHOR!,https://marketrealist.com/p/vethor-vtho-price-...,"[All hail VETHOR!, ]","[0.0, 0.0]","[0.557, 0.0]","[0.443, 0.0]","[0.1511, 0.0]",[0.1511],0.151100
2,cho0n22,1.628996e+09,p4lhcm,0.0,POLITICS,0.0,NaN,1.0,[],Are Crypto's High Trading Volumes A Scam?,https://finance.yahoo.com/m/e1945811-b6db-31eb...,"[Are Crypto's High Trading Volumes A Scam?, ]","[0.381, 0.0]","[0.619, 0.0]","[0.0, 0.0]","[-0.5719, 0.0]",[-0.5719],-0.571900
3,holyshithead,1.628995e+09,p4lg9e,1.0,SUPPORT,1.0,NaN,1.0,[I just got to thinking after hearing that Mar...,What if doge exists now only to ruin the crypt...,https://www.reddit.com/r/CryptoCurrency/commen...,[What if doge exists now only to ruin the cryp...,"[0.18, 0.0, 0.083, 0.135, 0.0, 0.0]","[0.711, 0.946, 0.917, 0.865, 0.803, 0.0]","[0.109, 0.054, 0.0, 0.0, 0.197, 0.0]","[-0.3612, 0.3612, -0.4019, -0.4215, 0.4019, 0.0]","[-0.3612, 0.3612, -0.4019, -0.4215, 0.4019]",-0.084300
4,Lj3380,1.628995e+09,p4lg3p,1.0,NaN,0.0,NaN,1.0,"[Slow and steady, it looks like BAT is coming ...",BAT Token,https://www.reddit.com/r/CryptoCurrency/commen...,"[BAT Token, Slow and steady, it looks like BAT...","[0.0, 0.0, 0.0, 0.0]","[1.0, 0.848, 0.811, 0.813]","[0.0, 0.152, 0.189, 0.187]","[0.0, 0.3612, 0.4215, 0.5362]","[0.3612, 0.4215, 0.5362]",0.439633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,DoeyB,1.628950e+09,p48jqt,1.0,SELF-STORY,0.0,NaN,1.0,[Just thought id like to make a friendly remin...,The importance of multiple emails,https://www.reddit.com/r/CryptoCurrency/commen...,"[The importance of multiple emails, Just thoug...","[0.0, 0.048, 0.0, 0.0, 0.0]","[0.615, 0.836, 1.0, 0.805, 0.885]","[0.385, 0.117, 0.0, 0.195, 0.115]","[0.3612, 0.8934, 0.0, 0.4404, 0.6652]","[0.3612, 0.8934, 0.4404, 0.6652]",0.590050
1030,kitisgreat,1.628950e+09,p48jqf,0.0,FINANCE,0.0,NaN,1.0,[],Surprising Investor: Intel Now Owns Coinbase S...,https://cryptopotato.com/surprising-investor-i...,[Surprising Investor: Intel Now Owns Coinbase ...,"[0.0, 0.0]","[0.538, 0.0]","[0.462, 0.0]","[0.5106, 0.0]",[0.5106],0.510600
1031,Caralynethegreat,1.628950e+09,p48is3,0.0,FINANCE,0.0,NaN,1.0,[],Nearly 30% of Canadians Are Cryptocurrency Inv...,https://cryptopotato.com/nearly-30-of-canadian...,[Nearly 30% of Canadians Are Cryptocurrency In...,"[0.0, 0.0]","[1.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]",[],0.000000
1032,dalinara,1.628950e+09,p48ijg,0.0,🔴 UNRELIABLE SOURCE,2.0,NaN,1.0,[],XRP chart triggers sell-off warning after pric...,https://cointelegraph.com/news/xrp-chart-trigg...,[XRP chart triggers sell-off warning after pri...,"[0.167, 0.0]","[0.833, 0.0]","[0.0, 0.0]","[-0.34, 0.0]",[-0.34],-0.340000


In [10]:
cats = texts[['link_flair_text','neg','neu','pos','comp_means']]
cats = pd.concat([cats.groupby(['link_flair_text']).mean(), cats.groupby(['link_flair_text']).std() ], axis=1)
cats

,comp_means,comp_means
link_flair_text,,
!= FORBES EDITORIAL,0.000000,0.000000
ADOPTION,0.362885,0.265671
CLIENT,0.282895,0.276433
COMEDY,0.124749,0.343097
CONTEST,0.685900,NaN
CREATIVE,0.325600,NaN
CRITICAL-DISCUSSION,0.218820,NaN
DEVELOPMENT,0.186377,0.403501
EDUCATIONAL,0.122579,0.469986


In [8]:
comms = texts[['num_comments','neg','neu','pos','comp_means']]
comms = pd.concat([comms.groupby(['num_comments']).mean(), comms.groupby(['num_comments']).std()], axis=1)
comms

,comp_means,comp_means
num_comments,,
0.0,0.233692,0.300705
1.0,0.182170,0.337642
2.0,0.170380,0.426930
3.0,0.182986,0.139365
16.0,0.138877,NaN


In [9]:
datlen = texts['data'].apply('.'.join).str.len()
bins = pd.IntervalIndex.from_tuples([(0, 20), (20, 50), (50, 75), (75, 100), (100, 150), (150, 200), (200, 300), (300, 500), (500, 1000), (1000, 2000), (2000, 3000), (3000, 10000)])
datlen = pd.cut(datlen, bins)
lens = pd.concat([texts[['neg','neu','pos','comp_means']], datlen], axis=1)
#lens
comms = pd.concat([lens.groupby(['data']).mean(), lens.groupby(['data']).std()], axis=1)
comms

,comp_means,comp_means
data,,
"(0, 20]",0.085011,0.248096
"(20, 50]",0.018548,0.309612
"(50, 75]",0.100315,0.314109
"(75, 100]",0.115300,0.380758
"(100, 150]",0.201670,0.378518
"(150, 200]",0.492893,0.310886
"(200, 300]",0.217122,0.375765
"(300, 500]",0.280218,0.334532
"(500, 1000]",0.249133,0.280075


In [ ]:
#